# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.126664e+02     1.505371e+00
 * time: 0.5942888259887695
     1     1.043365e+01     8.106226e-01
 * time: 2.127720832824707
     2    -1.064031e+01     9.781295e-01
 * time: 2.692888021469116
     3    -3.349271e+01     7.360967e-01
 * time: 3.580176830291748
     4    -4.641988e+01     5.530150e-01
 * time: 4.427206039428711
     5    -5.647682e+01     2.235053e-01
 * time: 5.2867348194122314
     6    -5.950394e+01     2.221215e-01
 * time: 5.875488996505737
     7    -6.077040e+01     8.791398e-02
 * time: 6.458319902420044
     8    -6.130689e+01     3.973034e-02
 * time: 7.0267088413238525
     9    -6.160766e+01     3.749805e-02
 * time: 7.5979509353637695
    10    -6.179503e+01     3.432408e-02
 * time: 8.216177940368652
    11    -6.195756e+01     2.662152e-02
 * time: 8.84922194480896
    12    -6.205672e+01     1.794362e-02
 * time: 9.458325862884521
    13    -6.211613e+01     1.472620e-02
 * time: 10.05177092552185
    

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671072
    AtomicLocal         -18.8557702
    AtomicNonlocal      14.8522663
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485376 
    Xc                  -19.3336822

    total               -62.261666461746
